In [21]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

## Prepare environment

In [8]:
torch.cuda.is_available()

False

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [10]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
# INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"   # home PC
INRIA_DATASET_PATH = "data/TestSubsets/INRIAAerialImageLabellingDataset"  # laptop

In [11]:
labeled_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, transforms=[ResizeToDivisibleBy32()]
)
print(len(labeled_dataset))

data/TestSubsets/INRIAAerialImageLabellingDataset\train
5


In [12]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, split="test", transforms=[ResizeToDivisibleBy32()]
)
print(len(test_dataset))

data/TestSubsets/INRIAAerialImageLabellingDataset\test
5


## Sanity check data

In [13]:
train_loader = DataLoader(labeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [16]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [17]:
to_pil_transform = transforms.ToPILImage()

In [18]:
img = to_pil_transform(images.squeeze())

In [23]:
# plt.imsave("assets/inria-example-image.jpeg", np.array(img))

In [19]:
# img.show()

In [24]:
msk = to_pil_transform(masks.squeeze())

In [25]:
# plt.imsave("assets/inria-example-mask.jpeg", np.array(msk))

In [ ]:
# msk.show()

# UNET

## Run example model on single image

In [ ]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
).to(device)

In [ ]:
baseline_model = BaselineModel()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    output_baseline = (output_baseline > 0.5).float()

In [ ]:
print(output.shape)

In [ ]:
print(output_baseline.shape)

In [ ]:
outp = to_pil_transform(output.squeeze().cpu().numpy())

In [ ]:
unique, counts = np.unique(outp, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
# outp.show()

In [ ]:
rand_outp = to_pil_transform(output_baseline.squeeze().cpu().numpy())

In [ ]:
unique, counts = np.unique(rand_outp, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
# rand_outp.show()

In [ ]:
evaluate_result(output, masks)

In [ ]:
evaluate_result(output_baseline, masks)

In [ ]:
evaluate_result(masks, masks)

## Run example model on dataset

In [ ]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [ ]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        output = (output > 0.5).float()

        iter_metrics = evaluate_result(output, masks)
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [ ]:
print(metrics_dict)

In [ ]:
print({key: np.mean(value) for key, value in metrics_dict.items()})

# UNET++

## Run example model on single image

In [26]:
model = smp.UnetPlusPlus(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
).to(device)

In [27]:
baseline_model = BaselineModel()

In [28]:
torch.cuda.empty_cache()

In [29]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [30]:
print(output.shape)

torch.Size([1, 1, 5024, 5024])


In [31]:
print(output_baseline.shape)

torch.Size([1, 1, 5024, 5024])


In [32]:
to_pil_transform = transforms.ToPILImage()

In [33]:
img = to_pil_transform(images.squeeze())

In [34]:
plt.imsave("assets/inria-example-image.jpeg", np.array(img))

In [35]:
# img.show()

In [36]:
msk = to_pil_transform(masks.squeeze())

In [37]:
plt.imsave("assets/inria-example-mask.jpeg", np.array(msk))

In [38]:
# msk.show()

In [39]:
outp = to_pil_transform(output.squeeze().cpu())

In [40]:
plt.imsave("assets/inria-example-unet-plus-plus-output.jpeg", np.array(outp))

In [ ]:
# outp.show()

In [42]:
evaluate_result(output, masks)

{'iou': tensor(5.0074e-06),
 'f1': tensor(1.0015e-05),
 'accuracy': tensor(0.8971),
 'recall': tensor(0.0024)}